In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from skimage import io

In [12]:
img_arr = io.imread('data/img.tif')

In [13]:
img_t = torch.tensor(img_arr, dtype = torch.float32)
img_t = torch.unsqueeze(torch.unsqueeze(img_t, 0), 0)
img_t.shape

torch.Size([1, 1, 128, 128])

Convolution

In [14]:
conv3 = nn.Conv2d(in_channels= 1, out_channels=64, kernel_size=7, stride=2, padding=3)

In [15]:
out1 = conv3(img_t)
out1.shape

torch.Size([1, 64, 64, 64])

Output Size = (Input Size + 2 * padding - Kernel Size)/Stride + 1

Batch Norm

In [16]:
bn = nn.BatchNorm2d(num_features= 64)
out2 = bn(out1)
out2.shape

torch.Size([1, 64, 64, 64])

Relu

In [17]:
out3 = F.relu(out2)
out3.shape

torch.Size([1, 64, 64, 64])

In [71]:
block = nn.Sequential(
    nn.Conv2d(64, 64, kernel_size= 3, stride = 1, padding = 1),
    nn.BatchNorm2d(num_features = 64),
    nn.ReLU())
block

Sequential(
  (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)

In [69]:
out4 = block(out3)
out5 = block(out)
out4.shape

torch.Size([1, 64, 64, 64])

First Block

Resnet Block 1

In [23]:
class ResBlock1(nn.Module):
    def __init__(self, channels):
        super(ResBlock1, self).__init__()
        self.channels = channels
        self.conv1 = nn.Conv2d(channels, channels, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = channels)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(num_features = channels)
        
    def forward(self, in_img):
        print(f'Input Shape {in_img.shape}')
        conv1_out = self.conv1(in_img)
        bn1_out = self.bn1(conv1_out)
        relu1_out = self.relu1(bn1_out)
        conv2_out = self.conv2(relu1_out)
        bn2_out = self.bn2(conv2_out)
        print(f'Output Shape {bn2_out.shape}')
        return in_img + bn2_out 

In [24]:
x = nn.Sequential(ResBlock1(64), ResBlock1(64), ResBlock1(64))

In [25]:
x = ResBlock1(64)
x(out3)

Input Shape torch.Size([1, 64, 64, 64])
Output Shape torch.Size([1, 64, 64, 64])


tensor([[[[ 0.7267, -0.2145,  0.4249,  ...,  0.8096,  0.6146,  0.1677],
          [ 0.2961,  0.0138,  0.1225,  ..., -0.2746,  0.3270,  0.0802],
          [ 1.1301,  0.1682, -0.6960,  ..., -0.9332, -0.2819,  0.2115],
          ...,
          [ 1.4692,  0.7054,  0.5997,  ...,  1.1866,  2.1095,  1.3435],
          [ 0.6519,  0.3783,  0.0155,  ...,  2.6122,  0.8732, -0.3146],
          [ 0.7320,  0.0673, -0.0047,  ...,  1.4980,  0.7517,  1.0803]],

         [[-0.1001,  0.6632,  0.2594,  ...,  0.0464,  0.5624,  0.1798],
          [-0.4820, -0.3353, -0.7902,  ..., -1.2012, -0.8175,  0.1098],
          [-0.6177,  0.4522, -0.9794,  ...,  0.5348, -1.0610, -0.0217],
          ...,
          [-0.1703,  0.9836, -0.1918,  ...,  0.0106,  0.0937,  1.3526],
          [-0.9071, -0.6552, -1.3168,  ...,  0.0098, -0.1875,  0.9162],
          [-0.2208,  0.7930, -0.2990,  ..., -0.0790, -1.4570, -0.8082]],

         [[-0.3207, -0.4826, -0.0786,  ..., -1.1759,  0.1550, -0.4832],
          [-0.2864,  0.3466, -

In [9]:
x

ResBlock1(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

Connection Block

In [59]:
class ConnBlock(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(ConnBlock, self).__init__()
        self.in_c = channels_in
        self.out_c = channels_out
        self.conv1 = nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 2, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = channels_out)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(channels_out, channels_out, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(num_features = channels_out)
        self.downsample = nn.Conv2d(channels_in, channels_out, kernel_size = 1, stride = 2)
        
    def forward(self, x):
        conv1_out = self.conv1(x)
        bn1_out = self.bn1(conv1_out)
        relu1_out = self.relu(bn1_out)
        conv2_out = self.conv2(relu1_out)
        bn2_out = self.bn2(conv2_out)
        x_downsample = self.downsample(x)
        out = self.relu(bn2_out + x_downsample)
        return out

In [60]:
x1 = ConnBlock(64, 128)
x1

ConnBlock(
  (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
)

In [61]:
x1(x(out3)).shape

Input Shape torch.Size([1, 64, 64, 64])
Output Shape torch.Size([1, 64, 64, 64])


torch.Size([1, 128, 32, 32])

In [53]:
conv1x1 = nn.Conv2d(64, 128, kernel_size = 1, stride = 2)

In [43]:
conv1x1(out3).shape

torch.Size([1, 128, 32, 32])

Joining all of this, by simple basic 